# Homework 2

**Name:** -- Agustin Villarreal --

**e-mail:** -- agustin.villarreal0743@alumnos.udg.mx --

# MODULES

In [5]:
# Load modules
import numpy as np
import math
import scipy.stats as stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import panel as pn
import param
import pandas as pd
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
import hvplot.pandas


In [7]:
# Enable Panel extensions
pn.extension('plotly')
hv.extension('plotly')


In [8]:
# Define Vec2d class for vector operations
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # String representation
    def __str__(self):
        return f"({self.x}, {self.y})"
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # Rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
    # Return as tuple
    def as_tuple(self):
        return (self.x, self.y)

# Randon Walk Trajectory Generation functions

In [9]:
# Brownian Motion
def generate_bm_trajectory(num_steps, speed, start_pos_x, start_pos_y, seed=None):
    """
    Generate a Brownian Motion trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    speed : float
        Speed/step size for the walker
    start_pos_x, start_pos_y : float
        Starting position coordinates
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_steps, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Initialize trajectory array
    trajectory = np.zeros((num_steps, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Store initial position
    trajectory[0, 0] = current_pos.x
    trajectory[0, 1] = current_pos.y
    trajectory[0, 2] = 0  # time
    
    # Generate trajectory
    for i in range(1, num_steps):
        # Sample random direction (uniform in all directions)
        angle = np.random.uniform(0, 2 * np.pi)
        
        # Create step vector
        step = Vec2d(np.cos(angle) * speed, np.sin(angle) * speed)
        
        # Update position
        current_pos = current_pos + step
        
        # Store position
        trajectory[i, 0] = current_pos.x
        trajectory[i, 1] = current_pos.y
        trajectory[i, 2] = i  # time
    
    return trajectory

In [10]:
# Create a Panel class for the dashboard
class RandomWalksDashboard(param.Parameterized):
    
    # Trajectory parameters
    rw_type = param.Selector(objects=["BM", "CRW", "LF"], default="BM")
    num_steps = param.Integer(default=1000, bounds=(100, 5000))
    speed = param.Number(default=5.0, bounds=(1.0, 20.0))
    start_pos_x = param.Integer(default=0, bounds=(-100, 100))
    start_pos_y = param.Integer(default=0, bounds=(-100, 100))
    cauchy_coef = param.Number(default=0.7, bounds=(0.1, 0.9))
    levy_exponent = param.Number(default=1.5, bounds=(1.1, 3.0))
    
    # Metric parameters
    metric_type = param.Selector(objects=["PL", "MSD", "TAD"], default="MSD")
    
    # Update button
    update_button = param.Action(lambda x: x.param.trigger('update_button'), label='Update')
    
    def __init__(self, **params):
        super(RandomWalksDashboard, self).__init__(**params)
        self.trajectory = None
        
    @param.depends('update_button', 'rw_type', watch=True)
    def generate_trajectory(self):
        """Generate a new trajectory based on current parameters"""
        
        # Common parameters for all trajectory types
        common_params = {
            'num_steps': self.num_steps,
            'speed': self.speed,
            'start_pos_x': self.start_pos_x,
            'start_pos_y': self.start_pos_y
        }
        
        # Generate trajectory based on selected type
        if self.rw_type == "BM":
            self.trajectory = generate_bm_trajectory(**common_params)
        elif self.rw_type == "CRW":
            self.trajectory = generate_crw_trajectory(
                **common_params, cauchy_coef=self.cauchy_coef)
        elif self.rw_type == "LF":
            self.trajectory = generate_lf_trajectory(
                **common_params, cauchy_coef=self.cauchy_coef, levy_exponent=self.levy_exponent)
        
        return self.trajectory
    
    def plot_trajectory(self):
        """Create a 3D plot of the trajectory"""
        if self.trajectory is None:
            self.generate_trajectory()
            
        # Create 3D trajectory plot
        fig = go.Figure()
        
        # Plot every nth point to improve performance
        plot_every_n = max(1, len(self.trajectory) // 500)
        
        fig.add_trace(go.Scatter3d(
            x=self.trajectory[::plot_every_n, 0],
            y=self.trajectory[::plot_every_n, 1],
            z=self.trajectory[::plot_every_n, 2],
            mode='lines',
            name=self.rw_type,
            line=dict(color='red', width=3)
        ))
        
        # Update layout
        fig.update_layout(
            title=f"{self.rw_type} trajectory",
            scene=dict(
                xaxis_title="x_pos",
                yaxis_title="y_pos",
                zaxis_title="time",
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=1)
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            template="plotly_white",
            height=500
        )
        
        return fig
    
    def calculate_metrics(self):
        """Calculate the selected metric for the current trajectory"""
        if self.trajectory is None:
            self.generate_trajectory()
        
        # Calculate the selected metric
        if self.metric_type == "PL":
            time, metric_values = calculate_pl(self.trajectory)
            xlabel, ylabel = "Time", "Path Length"
        elif self.metric_type == "MSD":
            time, metric_values = calculate_msd(self.trajectory)
            xlabel, ylabel = "tau", "MSD"
        elif self.metric_type == "TAD":
            time, metric_values = calculate_tad(self.trajectory)
            xlabel, ylabel = "Turning Angle (degrees)", "Probability Density"
        
        return time, metric_values, xlabel, ylabel
    
    def plot_metric(self):
        """Create a plot of the selected metric"""
        time, metric_values, xlabel, ylabel = self.calculate_metrics()
        
        # Create metric plot
        fig = go.Figure()
        
        # Add trace with color based on metric type
        color_map = {"PL": "green", "MSD": "purple", "TAD": "blue"}
        
        fig.add_trace(go.Scatter(
            x=time,
            y=metric_values,
            mode='lines',
            name=self.metric_type,
            line=dict(color=color_map[self.metric_type], width=2)
        ))
        
        # Update layout
        title_map = {
            "PL": "Path Length",
            "MSD": "Mean Squared Displacement",
            "TAD": "Turning Angle Distribution"
        }
        
        fig.update_layout(
            title=title_map[self.metric_type],
            xaxis_title=xlabel,
            yaxis_title=ylabel,
            template="plotly_white",
            margin=dict(l=0, r=0, b=0, t=30),
            height=500
        )
        
        return fig
    
    @param.depends('rw_type')
    def get_parameter_panel(self):
        """Return the appropriate parameter widgets based on RW type"""
        common_params = [
            pn.Param(self.param.num_steps, widgets={'num_steps': {'width': 150}}),
            pn.Param(self.param.speed, widgets={'speed': {'width': 150}}),
            pn.Param(self.param.start_pos_x, widgets={'start_pos_x': {'width': 150}}),
            pn.Param(self.param.start_pos_y, widgets={'start_pos_y': {'width': 150}}),
        ]
        
        if self.rw_type == "BM":
            return pn.Column(*common_params)
        elif self.rw_type == "CRW":
            return pn.Column(
                *common_params,
                pn.Param(self.param.cauchy_coef, widgets={'cauchy_coef': {'width': 150}}),
            )
        elif self.rw_type == "LF":
            return pn.Column(
                *common_params,
                pn.Param(self.param.cauchy_coef, widgets={'cauchy_coef': {'width': 150}}),
                pn.Param(self.param.levy_exponent, widgets={'levy_exponent': {'width': 150}}),
            )
    
    def view(self):
        """Create the dashboard layout"""
        
        # Initialize plots
        trajectory_plot = pn.pane.Plotly(self.plot_trajectory, height=500)
        metric_plot = pn.pane.Plotly(self.plot_metric, height=500)
        
        # Update plots when button is clicked
        def update_plots(event):
            trajectory_plot.object = self.plot_trajectory()
            metric_plot.object = self.plot_metric()
            
        self.param.update_button.watch(update_plots)
        
        # Create dashboard layout
        dashboard = pn.Column(
            pn.pane.Markdown("# Random Walks Dashboard", align='center'),
            pn.Row(
                # Left column - Controls
                pn.Column(
                    pn.pane.Markdown("## RW Type"),
                    pn.Param(self.param.rw_type, widgets={'rw_type': {'width': 150}}),
                    pn.pane.Markdown("## Parameters"),
                    self.get_parameter_panel,
                    pn.pane.Markdown("## Metric Type"),
                    pn.Param(self.param.metric_type, widgets={'metric_type': {'width': 150}}),
                    pn.Param(self.param.update_button, widgets={'update_button': {'width': 150, 'button_type': 'primary'}}),
                    width=250
                ),
                # Middle column - Trajectory
                pn.Column(
                    pn.pane.Markdown("## Trajectory"),
                    trajectory_plot,
                    width=500
                ),
                # Right column - Metric
                pn.Column(
                    pn.pane.Markdown("## Metric"),
                    metric_plot,
                    width=500
                )
            )
        )
        
        return dashboard

In [12]:
# Create and initialize the dashboard
dashboard = RandomWalksDashboard()

In [13]:
# Display the dashboard
dashboard.view().servable()

ValueError: Plotly pane does not support objects of type 'method'.